In [1]:
import pandas as pd
import numpy as np

# <span style="color:blue">Section1: Pretreatment of the data</span>

## First Dataframe corresponds to the original Dataset (34551849  records in total)

In [3]:
original_data = pd.read_csv('original_file.csv',sep='\t',header=None, names=['ID','DateTime','lon','lat'])
original_data

,ID,DateTime,lat,lon
0,1,2015-03-04 00:35:16,4.870147,45.772140
1,1,2015-03-04 00:35:48,4.870218,45.772095
2,1,2015-03-04 00:35:49,4.870210,45.772072
3,1,2015-03-04 00:35:50,4.870210,45.772072
4,1,2015-03-04 00:35:52,4.870210,45.772072
...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650
34551845,110,2015-03-12 16:23:22,2.343094,48.891650
34551846,110,2015-03-12 16:23:24,2.343094,48.891649
34551847,110,2015-03-12 16:23:25,2.343094,48.891649


## We will need the original number of records as parameter for some of the metrics

In [25]:
nb_original_lines = len(original_data)
nb_original_lines

34551849

## Second Dataframe corresponds to the anonymized Dataset WITHOUT the records marked as 'DEL'

In [27]:
anon_data = pd.read_csv('anon_data.csv',sep='\t',header=None, names=['ID','DateTime','lat','lon'])
anon_data = anon_data.loc[anon_data.ID !='DEL ']
anon_data

,ID,DateTime,lat,lon
0,404,2015-03-04 00:35:16,4.888,45.740
3,404,2015-03-04 00:35:50,4.867,45.750
4,404,2015-03-04 00:35:52,4.879,45.786
6,404,2015-03-04 00:35:55,4.873,45.785
7,404,2015-03-04 00:35:56,4.871,45.784
...,...,...,...,...
34551844,799,2015-03-12 16:23:21,4.872,45.783
34551845,799,2015-03-12 16:23:22,4.882,45.784
34551846,799,2015-03-12 16:23:24,4.873,45.785
34551847,799,2015-03-12 16:23:25,4.874,45.784


## Third Dataframe corresponds to the original dataframe except the records that were 'DEL' in the anonymized dataset 

### You should notice that the number of records in df_original_without_DEL and in anon_data should be the same

In [14]:
df_original_without_DEL = original_data.loc[anon_data.index]
df_original_without_DEL

,ID,DateTime,lat,lon
0,1,2015-03-04 00:35:16,4.870147,45.772140
3,1,2015-03-04 00:35:50,4.870210,45.772072
4,1,2015-03-04 00:35:52,4.870210,45.772072
6,1,2015-03-04 00:35:55,4.870210,45.772072
7,1,2015-03-04 00:35:56,4.870210,45.772072
...,...,...,...,...
34551844,110,2015-03-12 16:23:21,2.343094,48.891650
34551845,110,2015-03-12 16:23:22,2.343094,48.891650
34551846,110,2015-03-12 16:23:24,2.343094,48.891649
34551847,110,2015-03-12 16:23:25,2.343094,48.891649


# <span style="color:blue">Section 2: The Metrics</span>

## <span style="color:red">The Metrics ["utility_tuile", "utility_POI", "utility_meet"]: require the original df without removing the corresponding 'DEL' record in the anonymized dataset</span>

In [29]:
def POI_util(df_anon, df_original):
    #Global variables
    score = 0
    size = 2
    values = ['NIGHT', 'NIGHT','WORK', 'WEEKEND']
    df_orig = df_original.copy()
    df_anon = df_anon.copy()
    
    if(len(df_anon)!=0):
    
        df_orig.rename(columns={'id':'ID', 'date':'DateTime', 'latitude':'lat', 'longitude':'lon'}, inplace=True)
        df_anon.rename(columns={'id':'ID', 'date':'DateTime', 'latitude':'lat', 'longitude':'lon'}, inplace=True)

        df_orig['DateTime']= df_orig['DateTime'].astype('datetime64[ns]')
        df_orig['ID']= df_orig['ID'].astype('string')
        df_orig['lat']= df_orig['lat'].astype('float64')
        df_orig['lon']= df_orig['lon'].astype('float64')

        df_anon['DateTime']= df_anon['DateTime'].astype('datetime64[ns]')
        df_anon['ID']= df_anon['ID'].astype('string')
        df_anon['lat']= df_anon['lat'].astype('float64')
        df_anon['lon']= df_anon['lon'].astype('float64')
        #df_anon['ID']=df_orig['ID']
        df_anon['ID']= df_orig.loc[df_anon.index.intersection(df_orig.index),'ID']

        #Pre-treatment of original dataframe
        df_orig['lat']=df_orig['lat'].round(size)
        df_orig['lon']=df_orig['lon'].round(size)
        df_orig['Hour'] = df_orig.loc[:,'DateTime'].dt.hour
        df_orig['Day'] = df_orig.loc[:,'DateTime'].dt.day
        df_orig['Month'] = df_orig.loc[:,'DateTime'].dt.month
        df_orig['Week'] = df_orig.loc[:,'DateTime'].dt.isocalendar().week
        df_orig['DayOfTheWeek'] = df_orig.loc[:,'DateTime'].dt.dayofweek
        df_orig.sort_values(by=['ID', 'DateTime'], inplace=True)
        df_orig.reset_index(drop=True, inplace=True)
        df_orig['DatetimeIndex'] = np.select(conditions(df_orig), values, 'RegularTime')
        df_orig['time_spent']=0

        df_orig['Index_of_POI'] = df_orig['ID'] + '-' + df_orig['Day'].astype('string') + '-' + df_orig['Week'].astype('string') + '-' + df_orig['lat'].astype('string') + '-' + df_orig['lon'].astype('string') + '-' + df_orig['DatetimeIndex'].astype('string')
        df_orig['Index_of_POI_shifted_backward'] = df_orig['Index_of_POI'].shift(-1)
        df_orig['Index_of_POI_shifted_forward'] = df_orig['Index_of_POI'].shift(+1)
        df_orig.loc[0,'Index_of_POI_shifted_forward']='0'
        df_orig.loc[len(df_orig)-1,'Index_of_POI_shifted_backward']='0'
        df_orig['start_time'] = df_orig.loc[~(df_orig['Index_of_POI']==df_orig['Index_of_POI_shifted_forward']), 'DateTime']
        df_orig.fillna(method="ffill", inplace=True)
        df_orig['time_spent'] = (df_orig['DateTime'] - df_orig['start_time'])

        #Getting the POI
        df_orig2 = df_orig.loc[~(df_orig['Index_of_POI_shifted_backward']==df_orig['Index_of_POI']),['ID', 'lat', 'lon', 'Week', 'DatetimeIndex', 'time_spent']].groupby(by=['ID', 'lat', 'lon', 'Week', 'DatetimeIndex']).sum().reset_index()
        df_orig2 = df_orig2.sort_values(by=['ID', 'Week', 'time_spent',  'DatetimeIndex'], ascending=[True, True, False, False]).reset_index(drop=True)
        df_orig2 = df_orig2.groupby(by=['ID', 'Week', 'DatetimeIndex']).head(1).reset_index(drop=True)


        #Pre-treatment of anonymized dataframe

        df_anon['lat']=df_anon['lat'].round(size)
        df_anon['lon']=df_anon['lon'].round(size)
        df_anon['Hour'] = df_anon.loc[:,'DateTime'].dt.hour
        df_anon['Day'] = df_anon.loc[:,'DateTime'].dt.day
        df_anon['Month'] = df_anon.loc[:,'DateTime'].dt.month
        df_anon['Week'] = df_anon.loc[:,'DateTime'].dt.isocalendar().week
        df_anon['DayOfTheWeek'] = df_anon.loc[:,'DateTime'].dt.dayofweek
        df_anon.sort_values(by=['ID', 'DateTime'], inplace=True)
        df_anon.reset_index(drop=True, inplace=True)
        df_anon['DatetimeIndex'] = np.select(conditions(df_anon), values, 'RegularTime')
        df_anon['time_spent']=0

        df_anon['Index_of_POI'] = df_anon['ID'] + '-' + df_anon['Day'].astype('string') + '-' + df_anon['Week'].astype('string') + '-' + df_anon['lat'].astype('string') + '-' + df_anon['lon'].astype('string') + '-' + df_anon['DatetimeIndex'].astype('string')
        df_anon['Index_of_POI_shifted_backward'] = df_anon['Index_of_POI'].shift(-1)
        df_anon['Index_of_POI_shifted_forward'] = df_anon['Index_of_POI'].shift(+1)
        df_anon.loc[0,'Index_of_POI_shifted_forward']='0'
        df_anon.loc[len(df_anon)-1,'Index_of_POI_shifted_backward']='0'
        df_anon['start_time'] = df_anon.loc[~(df_anon['Index_of_POI']==df_anon['Index_of_POI_shifted_forward']), 'DateTime']
        df_anon.fillna(method="ffill", inplace=True)
        df_anon['time_spent'] = (df_anon['DateTime'] - df_anon['start_time'])

        df_anon2 = df_anon.loc[~(df_anon['Index_of_POI_shifted_backward']==df_anon['Index_of_POI']),['ID', 'lat', 'lon', 'Week', 'DatetimeIndex', 'time_spent']].groupby(by=['ID', 'lat', 'lon', 'Week', 'DatetimeIndex']).sum().reset_index()
        df_anon2 = df_anon2.sort_values(by=['ID', 'Week', 'time_spent',  'DatetimeIndex'], ascending=[True, True, False, False]).reset_index(drop=True)

        #Comparing the time spent in POI between original and anonymized dataset
        df_orig2 = df_orig2.loc[~(df_orig2.DatetimeIndex =='RegularTime')]
        left_join_df = pd.merge(df_orig2, df_anon2, on=['ID','lat','lon','Week','DatetimeIndex'], how='left')

        left_join_df['time_spent_y'] = left_join_df['time_spent_y'].fillna(pd.Timedelta(seconds=0))
        left_join_df['diff_time_spent'] = abs( left_join_df['time_spent_y'].dt.total_seconds() - left_join_df['time_spent_x'].dt.total_seconds() )
        left_join_df['time_spent_x'] = left_join_df['time_spent_x'].dt.total_seconds()
        
        #Calculating the scrore
        score = 1- (left_join_df['diff_time_spent'].sum()/left_join_df['time_spent_x'].sum())
    return score

def conditions(df):
    return [
        (df['DayOfTheWeek'] < 4) & (df['Hour']>=22) & (df['Hour']<=23), 
        (df['DayOfTheWeek'] < 4) & (df['Hour']>=0) & (df['Hour']<=6),
        (df['DayOfTheWeek'] < 4) & (df['Hour']>=9) & (df['Hour']<=17),
        (df['DayOfTheWeek'] >= 4) & (df['Hour']>=10) & (df['Hour']<=18)
    ]



In [ ]:
# notice the argument we are using the original_data with the anonymized_data (different dataframe sizes)
print("POI Utility Score =",POI_util(anon_data, original_data))

### nothing has changed in the meet_utility metric

In [32]:
def meet_utility(df_anon, df_original):
    #Define global variable
    size = 2
    pt = 0.1

    df = df_anon.copy()
    df_orig = df_original.copy()

    # Converting longitude and latitude as float 
    df = df.astype({'lon': 'float64', 'lat': 'float64'})
    df_orig = df_orig.astype({'lon': 'float64', 'lat': 'float64'})

    # Round lat,long with size
    df['lat'] = df['lat'].round(size)
    df['lon'] = df['lon'].round(size)
    df_orig['lat'] = df_orig['lat'].round(size)
    df_orig['lon'] = df_orig['lon'].round(size)

    # get all unique positions and sort them by most visited
    df = df.groupby(['lat','lon']).size().reset_index(name='count')
    df_orig = df_orig.groupby(['lat','lon']).size().reset_index(name='count')
    df = df.sort_values(by=['count'])
    df_orig = df_orig.sort_values(by=['count'])

    # Only keep top % cells
    nb_cellules = int(len(df_orig)*pt)
    df = df.head(nb_cellules)
    df_orig = df_orig.head(nb_cellules)

    # left join and compare cells
    df = pd.merge(df_orig, df, on=['lat', 'lon'], how='left')
    
    score = df['count_y'].notnull().sum()
    return score / nb_cellules


#### notice the argument we are using the original_data with the anonymized_data (different dataframe sizes)

In [ ]:
print("Meet Utility score =", meet_utility(anon_data, original_data))

### nothing has changed in the tuile_utility metric

In [31]:
def tuile_utility(df_anon, df_original):
    size = 2
    df = df_anon.copy()
    df_orig = df_original.copy()
    
    # Converting longitude and latitude as float 
    df = df.astype({'lon': 'float64', 'lat': 'float64', 'ID': 'string' })
    df_orig = df_orig.astype({'lon': 'float64', 'lat': 'float64', 'ID': 'string'})

    # Round lat,long with size
    df['lat'] = df['lat'].round(size)
    df['lon'] = df['lon'].round(size)
    df_orig['lat'] = df_orig['lat'].round(size)
    df_orig['lon'] = df_orig['lon'].round(size)

    # Group each position for ids and retrieve the count of unique position
    df = df.groupby(['ID','lat','lon']).size().reset_index(name='count')
    df_orig = df_orig.groupby(['ID','lat','lon']).size().reset_index(name='count')
    df = df.groupby(['ID']).size().reset_index(name='count')
    df_orig = df_orig.groupby(['ID']).size().reset_index(name='count')

    df = pd.merge(df_orig, df, on=['ID'], how='left')
    df['score'] = df.apply(createScore, axis=1)
    score = df['score'].sum()
    return score / len(df)

def createScore(row):
    if pd.isnull(row['count_x']) or pd.isnull(row['count_y']):
        score = 0
    elif row['count_x'] > row['count_y']:
        score = row['count_y'] / row['count_x']
    else:
        score = row['count_x'] / row['count_y']
    return score


### notice the argument we are using the original_data with the anonymized_data (different dataframe sizes)

In [ ]:
print("Tuile Utility score =", tuile_utility(anon_data, original_data))

## <span style="color:red">The Metrics ["hour_utility", "date_utility", "distance_util"]: compare the datasets on only the records that were not deleted. yet to compute the mean of the score of each row we divide by the "nb_original_lines" which will take into consideration a utility =0 for the records that were deleted in the numerator</span>

### hour_utility Update: to take into consideration the DEL records with score =0 
We divide by the original number of records thus making date_util = 0 for the 'DEL' records

In [1]:
def hour_utility(df_anon, df_original, nb_original_lines):
    df_orig = df_original.copy()
    df_anon = df_anon.copy()

    df = pd.DataFrame({ 'df_hour': df_anon['DateTime'].dt.hour, 'df_origin_hour': df_orig['DateTime'].dt.hour })
    #Chaque ligne vaut 1 point
    #Une fraction de point eguale a 1/24 est enlevée à chaque heure d'écart
    df['hour_util'] = 1- abs(df['df_hour'] - df['df_origin_hour'])/24
    # le score finale est la moyenne d'ecart d'heures sur tous les points detecter
    score_hour_utility = df["hour_util"].sum()/nb_original_lines
    
    return score_hour_utility

In [ ]:
# notice the argument we are using the df_original_without_DEL with the anonymized_data (same dataframe sizes)
print("Hour Utility score =", hour_utility(anon_data, df_original_without_DEL, nb_original_lines))

### date_utility Update: to take into consideration the DEL records with score =0 
We divide by the original number of records thus making date_util = 0 for the 'DEL' records

In [ ]:
def date_utility(df_anon, df_original, nb_original_lines):
    df_orig = df_original.copy()
    df_anon = df_anon.copy()
#     df_orig.reset_index(drop=True, inplace=True)
#     df_anon.reset_index(drop=True, inplace=True)

    df_date_utility = pd.DataFrame({ 'DayOfTheWeek_orig': df_orig['DateTime'].dt.dayofweek, 'DayOfTheWeek_anon': df_anon['DateTime'].dt.dayofweek, 'Week_orig':df_orig['DateTime'].dt.isocalendar().week, 'Week_anon': df_anon['DateTime'].dt.isocalendar().week })
    df_date_utility['DiffDate'] = abs(df_date_utility['DayOfTheWeek_orig']-df_date_utility['DayOfTheWeek_anon'])
    #pour tout changement de semaine l'utilite doit etre 0 
    df_date_utility.loc[~(df_date_utility['Week_orig']==df_date_utility['Week_anon']),'DiffDate']=7
    df_date_utility['date_util']= 1- df_date_utility['DiffDate']/7
    score = df_date_utility["date_util"].sum()/nb_original_lines
    return score

#### notice the argument we are using the df_original_without_DEL with the anonymized_data (same dataframe sizes)

In [39]:
print("Date Utility score =",Date_utility(anon_data, df_original_without_DEL, nb_original_lines))

### distance_utility Update: to take into consideration the DEL records with score =0 
We set: </br> 'Distance_Accuracy' = 1 (for any record whose Haversine distance <1 km) </br>
       'Distance_Accuracy' = 1/Haversine_distance (for the records where the distance > 1 km)</br>
       The score is the sum of the 'Distance_Accuracy' but divided by the original number of records (thus considering their     distance _accuracy =0)

In [ ]:
def distance_utility(df_anon, df_original, nb_original_lines):

    df_orig = df_original.copy()
    df_anon = df_anon.copy()
#     df_orig.reset_index(drop=True, inplace=True)
#     df_anon.reset_index(drop=True, inplace=True)
    
    df_anon.rename(columns={'ID':'ID_ano', 'DateTime':'DateTime_ano', 'lat':'lat_ano', 'lon':'lon_ano'}, inplace = True)
    df = pd.concat([df_orig.reset_index(drop=True),df_anon.reset_index(drop=True)], axis=1)
    
    #Haversine distance
    to_radians = np.pi /180
    R = 6371.009 #en km
    #a=np.sin(((df.lat*to_radians-df.lat_ano*to_radians)/2)**2) + np.sin((((df.lon*to_radians-df.lon_ano*to_radians)/2)**2))*np.cos(df.lat*to_radians)*np.cos(df.lat_ano*to_radians)
    a = np.sin(((df.lat*to_radians-df.lat_ano*to_radians)/2))**2 + (np.sin((((df.lon*to_radians-df.lon_ano*to_radians)/2)))**(2))*np.cos(df.lat*to_radians)*np.cos(df.lat_ano*to_radians)
    b = np.sqrt(a)
    df['Haversine_Distance']= 2 * R * np.arcsin(b)
    
    df['Distance_Accuracy'] = 1 #cette valeur reste valide pour tout changement dans un rayon de 1Km
    df.loc[df['Haversine_Distance'] >1 , 'Distance_Accuracy'] = 1/df['Haversine_Distance']
    score = df["Distance_Accuracy"].sum()/nb_original_lines
    return score


#### notice the argument we are using the df_original_without_DEL with the anonymized_data (same dataframe sizes)

In [ ]:
# notice the argument we are using the df_original_without_DEL with the anonymized_data (same dataframe sizes)
print("Distance Utility score =",distance_utility(anon_data, df_original_without_DEL, nb_original_lines))